<a href="https://colab.research.google.com/github/briannamathre/LupusResearch/blob/master/LupusResearchScripts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here I am filitering the dataset to have the desired columns and the correct 
context. The filtered dataframe will then be writen to a file named "filtered_data.csv". 

In [ ]:
import pandas as pd
import os

#Read in the original CSV
df = pd.read_csv('filtered_data_unprocessed.csv') 

#Only keep the following columns from the dataframe
df = df[['SNPS','MAPPED_GENE','DOWNSTREAM_GENE_ID','UPSTREAM_GENE_ID','SNP_GENE_IDS','CONTEXT','MAPPED_TRAIT', 'P-VALUE']]

#Ensure that we are only keeping the SNPs with the following context:
#This command will return TRUE or FALSE whether the column contains these values
list_Valid_Context = ['intergenic_variant','3_prime_UTR_variant','5_prime_UTR_variant','intron_variant','regulatory_region_variant','TF_binding_site_variant']

#Filter through the dataframe according to the filter. 
df = df[df.CONTEXT.isin(list_Valid_Context)]

#Send our new-filtered data to a new csv file. 
df.to_csv('filtered_data.csv', index=False)


To get the number of variants

In [ ]:
#This will print the number of variants that we will actually be analyzing. 
len(df)

682

This code will allow me to put all of the rsIDs into a list. After the list is created all of these will be added into a set so that only unique values are counted. The set will then be converted back into a list so that further analysis can be made.

In [ ]:
#Here we will take the SNPs column and add it to a list.
contextList = df.SNPS.tolist()

#Make sure the length observed here was the same length as we found before.
print(len(contextList))

#Change the list into a set to get rid of duplicates
contextList= list(set(contextList))

#Record the number of non-duplicate variants. 
print(len(contextList))

682
521


This code will create text files that have 100 SNPS in them. This is becuase the tool PhenolScanner can only accept 100 SNPs at a time.

In [ ]:
import pandas as pd
import os

# We will loop through the data 5 times.
for i in range((int(len(contextList)/100))):
  start = i * 100 #Calculate the starting point
  end = ((i + 1) * 100) #Calculate the ending point
  #Create and open a new text flle
  f = open(str(start) + "_" + str(end) + ".txt", "a")
  #Parse through the list using the calculated start/end points
  newList=contextList[start:end]
  #Using each rsID, make sure that it begins with rs and if it does,
  # write it to the file
  for item in newList:
    x = item.startswith("rs")
    if x == True:
      f.write(item + "\n")
    else: 
      pass
  #After we have finished adding all of the rsIDs to the file, close it.
  f.close()

#This is the final text file, which does not contain 100 SNPSs
f = open('500_end.txt', "a")
#Get the final 21 rsIDs in the list
newList=contextList[500:]
for item in newList:
    x = item.startswith("rs")
    if x == True:
      f.write(item + "\n")
    else: 
      pass
f.close()

Double check to make sure each file has 100 or SNPS in it.

In [ ]:
fileNames = ["0_100.txt","100_200.txt","200_300.txt","300_400.txt","400_500.txt","500_end.txt"]
all_rsIDs = []
for files in fileNames:
  fileName = open(files, "r")
  fileItems = 0
  for line in fileName:
    all_rsIDs.append(line)
    fileItems += 1
  print(fileItems)

rsIDs = pd.DataFrame(all_rsIDs)
print(len(rsIDs))
rsIDs.to_csv("allRSids.csv")

100
100
100
100
100
21
521


This file will take all of the UP_STREAM genes, the DOWN_STREAM genes
and SNP_Gene Ids and will put them all into one list. After this list
is created, duplicates will be removed, and then each gene will be written
on a new line in a text file. There were 525 unique genes identified

In [ ]:
import pandas as pd
import os

SNP_Gene_List = df.SNP_GENE_IDS.tolist()
Down_Stream_Genes = df.DOWNSTREAM_GENE_ID.tolist()
Up_Stream_Genes = df.UPSTREAM_GENE_ID.tolist()
print(len(SNP_Gene_List))
print(len(Down_Stream_Genes))
print(len(Up_Stream_Genes))
SNP_Gene_List.extend(Down_Stream_Genes)
SNP_Gene_List.extend(Up_Stream_Genes)
print(len(SNP_Gene_List))

f = open("AllGenes.txt", "a")
geneList = []
for item in SNP_Gene_List:
  if str(item) == 'nan':
    pass
  else: 
    if (len(str(item)) > 16):
      gene_IDS = str(item)
      gene_IDS = gene_IDS.split(", ")
      for thing in gene_IDS:
        geneList.append(str(thing))
    else: 
      geneList.append(str(item))

f = open("AllGenes.txt", "a")
geneList = list(set(geneList))
print(len(geneList))
for gene in geneList:
  f.write(str(gene) + "\n")
f.close()




682
682
682
2046
525


Here we will take the eQTL results files and then combine them all into one file. 
We will take all of the results and extract the expressed gene based off of the rsID. We will then check to see if that gene has already been considered. If it has, then we skip it, otherwise we add that row to our "total_genes.csv".

In [ ]:
import pandas as pd
import os
#You will need to download these files from PhenoScanner after you have ran the analysis.
files = ["0_100_PhenoScanner_eQTL.csv","100_200_PhenoScanner_eQTL.csv","200_300_PhenoScanner_eQTL.csv","300_400_PhenoScanner_eQTL.csv","400_500_PhenoScanner_eQTL.csv","500_end_PhenoScanner_eQTL.csv"]

results = pd.DataFrame()

#Combine all of the results from all of the files into one file
for file1 in files:
  df = pd.read_csv(file1) 
  results = results.append(df)

#Now sort those combined files
final_results = results.sort_values(by=["p"])
final_results.to_csv("eQTL_Combined.csv")

res = [] 
total_unique = []
#Here we will find the number of unique genes expressed across all of the files
for index, row in final_results.iterrows():
    expressed_gene = row['exp_gene']
    if expressed_gene not in res: 
      res.append(expressed_gene)
      total_unique.append(row)
    
total_unique = pd.DataFrame(total_unique)

total_unique.to_csv("total_unique_genes.csv")
print(len(total_unique))







2659


Here we will take the total unique genes, and I will pull out all of the rsIds from the original list. We will take the eQTL results that have all of the total unique genes. We will use all of the known rsIDs from the original data and will compare them against the eQTL results. This program will return the number of rsIDs that were found in both the rsIDS from the GWAS catalog and from the eQTL results. 

In [1]:
import pandas as pd
import os

eqtl_results = pd.read_csv("total_unique_genes.csv") 

#The "allRSids.csv" file can be found in the Original_Data folder on Github
rsids = pd.read_csv("allRSids.csv")
rsids = rsids["0"].to_list()
clean_rsIDs = []
for rsID in rsids:
  rsID = rsID[:-1]
  clean_rsIDs.append(rsID)

total_found = []
for index, row in eqtl_results.iterrows():
    rsid = row["rsid"]
    if rsid in clean_rsIDs:
      total_found.append(row)

total_found = pd.DataFrame(total_found)
total_found = total_found["rsid"]

total_found_set=set(total_found)
print(total_found_set)
print(len(total_found_set))
results = []
final_results = []
for index, row in eqtl_results.iterrows():
  for r in total_found:
    rsid = row["rsid"]
    if (rsid not in final_results) and rsid == r:
      results.append(row)
      final_results.append(r)

print(len(results))
results = pd.DataFrame(results)
results.to_csv("eQTL_genes_found.csv")




{'rs2041670', 'rs150518861', 'rs4958880', 'rs4748857', 'rs10946940', 'rs79404002', 'rs6871748', 'rs351758', 'rs2275247', 'rs427221', 'rs6804441', 'rs58688157', 'rs1405209', 'rs2688608', 'rs12736195', 'rs2238573', 'rs12664535', 'rs11697848', 'rs6679677', 'rs17083844', 'rs1150754', 'rs7197475', 'rs3087243', 'rs835573', 'rs2431099', 'rs2187668', 'rs6601327', 'rs58721818', 'rs11154801', 'rs4963128', 'rs1048257', 'rs10276619', 'rs4795774', 'rs5986948', 'rs7579944', 'rs564799', 'rs5754467', 'rs11887156', 'rs931127', 'rs6538678', 'rs2111485', 'rs8058578', 'rs869310', 'rs558702', 'rs17552904', 'rs2303745', 'rs131654', 'rs3794060', 'rs2396545', 'rs3129716', 'rs1217393', 'rs11264750', 'rs1170426', 'rs2015407', 'rs180977001', 'rs1167796', 'rs11085727', 'rs6590330', 'rs10142203', 'rs1590381', 'rs4690055', 'rs9394274', 'rs7329174', 'rs1780813', 'rs11724582', 'rs11150610', 'rs4252665', 'rs12971295', 'rs11117433', 'rs12565776', 'rs1946007', 'rs1535001', 'rs1385374', 'rs564976', 'rs10889681', 'rs76725

Here we will take the Case vs. Control studies and extract the columns with the gene name. The top 1000 will be taken from each file and combined into one master file. The top 1000 genes will be taken. 

In [ ]:
import pandas as pd
import os
#You will need to run the analyses on GEO2R based off of given parameters in the 
# Materials/Methods section. The results will need to be downloaded from GEO2R
files = ['GSE11907.top.table.csv',"GSE10325.top.table.csv","GSE39088.top.table.csv",
         "GSE45291.top.table.csv","GSE49454.top.table.csv","GSE51997.top.table.csv"]

case_vs_control = pd.DataFrame()
for file1 in files:
  df = pd.read_csv(file1) 
  #Take the top 4000 results
  df = df[:4000]
  case_vs_control = case_vs_control.append(df)

#Take all of the results, sort them by p-value, and then add all of the results
# to a new file with the combined results. 
final_cvc_results = case_vs_control.sort_values(by=["P.Value"])
final_cvc_results.to_csv("CVC_Combined.csv")




Now we will take the combined results table and get all of the unique gene names, in order. 


In [ ]:
import pandas as pd
import os

results = pd.read_csv("CVC_Combined.csv") 


final_results = results.sort_values(by=["adj.P.Val"])

exp_genes = final_results['Gene.symbol']

res = [] 
for i in exp_genes: 
    if i not in res and str(i) != "nan": 
        res.append(i) 

res = res[:1000]
caseVcontrol_data = pd.DataFrame(res)
caseVcontrol_data.to_csv("Top_1000_results.csv")



Now I will take the KEGG pathway enriched table and separate the userID's so that they have their associated p-value which each gene. 


---



***DO WE USE THIS??***

---



In [ ]:
import pandas as pd
import os
#Here you will need to get these results, using the specified settings,
# and by downloading and using the given file. 
enrichment = pd.read_csv("enrichment_results_wg_result1614031939.csv") 


dic = dict()

#key as existing values and value as the p-value
for index, row in enrichment.iterrows():
    pValue = row["pValue"]
    values = []
    values.append(pValue)
    keys = row["userId"].split(";")
    for key in keys:
      if key not in dic:
        dic[key] = values
      else: 
        continue
        #we can use this code if we want ALL of the pvalues they were associated with, 
        #   not just the lowest p-Value
        #items = list(dic[key])
        #items.append(pValue)
        #dic[key] = items

df = pd.DataFrame(list(dic.items()),columns = ['EnsemblID','pValue']) 

df.to_csv("enrichment_ensembl_pvalues.csv")





Here is to get the top 10 KEGG pathways

In [ ]:
import pandas as pd
import os
#To get these file, you should run WebGestalt using the specified settings,
# and by downloading the file that begins with "enrichment_results_wg_".
# After downloading this file it was renamed. 
enrichment_KEGG = pd.read_csv("KEGG_Initial_Results.csv")
enrichment_KEGG = enrichment_KEGG[["geneSet","description","size","expect", "pValue","FDR"]]
top_20_results_KEGG = []
for index, row in enrichment_KEGG.iterrows():
  top_20_results_KEGG.append(row)
  
print(len(top_20_results_KEGG))
top_20_results_KEGG = pd.DataFrame(top_20_results_KEGG[:20])

top_20_results_KEGG.to_csv("KEGG_Top_20.csv")




37


Here is to get the top 20 Biological Processes

In [ ]:
import pandas as pd
import os
enrichment_biological = pd.read_csv("Biological_Processes_Initial_Results.csv")

enrichment_biological = enrichment_biological[["geneSet","description","size","expect", "pValue","FDR"]]
top_20_results_biological = []
for index, row in enrichment_biological.iterrows():
  top_20_results_biological.append(row)

print(len(top_20_results_biological))
    
top_20_results_biological = pd.DataFrame(top_20_results_biological[:20])

top_20_results_biological.to_csv("Biological_Processes_Top_20.csv")




362


Here is the code for the Top 20 cellular components

In [ ]:
import pandas as pd
import os
enrichment_cellular = pd.read_csv("Cellular_Components_Initial_Results.csv")

enrichment_cellular = enrichment_cellular[["geneSet","description","size","expect", "pValue","FDR"]]
top_20_results_cellular = []
for index, row in enrichment_cellular.iterrows():
  top_20_results_cellular.append(row)
    
print(len(top_20_results_cellular))
top_20_results_cellular = pd.DataFrame(top_20_results_cellular[:20])

top_20_results_cellular.to_csv("Cellular_Components_Top_20.csv")

43


Here is the code for the Top 20 molecular functions

In [ ]:
import pandas as pd
import os
enrichment_molecular = pd.read_csv("Molecular_Function_Initial_Results.csv")

enrichment_molecular = enrichment_molecular[["geneSet","description","size","expect", "pValue","FDR"]]
top_20_results_molecular = []
for index, row in enrichment_molecular.iterrows():
  top_20_results_molecular.append(row)
    
print(len(top_20_results_molecular))
top_20_results_molecular = pd.DataFrame(top_20_results_molecular[:20])

top_20_results_molecular.to_csv("Molecular_Functions_Top_20.csv")

40


Here I will take all of the identified risk genes, and see how many of those were in the CVC Dataset

In [ ]:
import pandas as pd
import os

mapped_genes_df = pd.read_csv("filtered_data.csv")
individual_genes = []
for index, row in mapped_genes_df.iterrows():
    genes = row["MAPPED_GENE"].split(",")
    for item in genes:
      secondSplit = item.split("-")
      for gene in secondSplit:
          if ("." not in gene) and (gene not in individual_genes):
            individual_genes.append(gene)
print(individual_genes)

#the list individual genes has all of the mapped gene names from the original snps
CVC_data = pd.read_csv("CVC_Combined.csv")

risk_genes_in_CVC = []
total_genes_in_CVC = set()
rows = []
for index, row in CVC_data.iterrows():
  expressed_gene = row['Gene.symbol']
  total_genes_in_CVC.add(expressed_gene)
  if expressed_gene in individual_genes:
      risk_genes_in_CVC.append(expressed_gene)
      rows.append(row)

CVC_results = pd.DataFrame(rows)

CVC_results.to_csv("CVC_risk_genes.csv")
CVC_results = CVC_results.sort_values(by=["adj.P.Val"])
print(len(total_genes_in_CVC))
print(len(risk_genes_in_CVC))
print(risk_genes_in_CVC)
      






['STAT4', 'TNPO3', 'BLK', ' IRF5', 'XKR6', 'GLT1D1', 'MTCO3P1 ', ' SNRPC', 'CD83P1 ', ' RNU6', '471P', 'LAMC1 ', ' LAMC2', ' EDEM3', 'TEX51 ', ' RNU7', '182P', 'RNU6', '546P ', 'CNTN6 ', ' RPL23AP38', 'C2', ' C2', ' TNPO3', 'MTG1', ' APIP', 'FAM98B', 'RN7SKP181 ', ' LINC02253', 'ITGAM', 'MED1 ', ' CDK12', ' RASSF2', 'KRT18P4 ', '147P', 'HLA', 'DQB2', ' HLA', 'TNFAIP3', 'WDFY4', ' RPL7P13', 'COX10', 'AS1', 'TNXB', ' TNXB', 'SLC1A7 ', ' LINC00578', 'TMC2', ' BLK', 'LAMC2', 'UBE2L3', 'RNA5SP73 ', ' LINC01701', 'DQA1', 'RCC2P8 ', ' COL25A1', 'MIR3142HG', 'PHRF1', ' PHRF1', 'TSBP1', 'NAALADL2', 'AS2', ' NAALADL2', 'DQB1 ', ' MTCO3P1', 'ELF1', 'ETS1', ' GHR', 'NEGR1', ' NEGR1', 'IT1', 'NTNG2', 'MSH5', ' MSH5', 'SAPCD1', 'RNU6ATAC13P ', ' HIVEP3', 'CRB1', 'LNC', 'LBCS', ' RPL35AP19', 'PRPF18', 'CCDC38', ' SNRPF', 'NIN', 'SLC5A11', 'ARMH4', 'ANKRD44', 'PRKG1', 'LINC01624 ', ' DLL1', ' RN7SKP82', 'CHIA', 'DRD2 ', ' TMPRSS5', ' ANKRD44', 'SCARB1', 'LINC02835 ', ' EPHA5', 'CAMK1D', 'BARX2', 'LINC

To get the adj.p.values under 1E-5

In [ ]:
import pandas as pd
import os

CVC_risk_genes = pd.read_csv("CVC_risk_genes.csv")
CVC_risk_genes_list = []
rows = []
for index, row in CVC_risk_genes.iterrows():
  gene_symbol = row["Gene.symbol"]
  p_value = row["adj.P.Val"]
  if p_value < 1E-5:
    CVC_risk_genes_list.append(gene_symbol)
    rows.append(row)


#CVC_risk_genes_list = CVC_risk_genes['Gene.symbol'].to_list()
rows_df = pd.DataFrame(rows)
rows_df.to_csv("CVC_genes_52.csv")

Here I will attempt the comparative analysis.


In [ ]:
import pandas as pd
import os
eQTL_genes = pd.read_csv("eQTL_Combined.csv")

"""res = [] 
top_100_results = []
for index, row in eQTL_genes.iterrows():
    expressed_gene = row['exp_gene']
    if expressed_gene not in res: 
      res.append(expressed_gene)
      top_100_results.append(row)
    
top_100_results = pd.DataFrame(top_100_results[:1000])
top_100_results.to_csv("top_1000_eQTL.csv")
"""

CVC_genes = pd.read_csv("Top_1000_results.csv")
eQTL_genes = pd.read_csv("eQTL_genes_found.csv")

CVC_gene_set = set(CVC_genes["0"].tolist())
eQTL_gene_set = set(eQTL_genes["exp_gene"].tolist())

CVC_data = pd.read_csv("CVC_Combined.csv")

#Check this
list_of_overlap = CVC_gene_set.intersection(eQTL_gene_set)
print(len(list_of_overlap))
print(list_of_overlap)
#

final_results = []
for gene_name in list_of_overlap:
  for index, row in CVC_data.iterrows():
    expressed_gene = row['Gene.symbol']
    if gene_name == expressed_gene:
      final_results.append(row)

final_results_df = pd.DataFrame(final_results)
print(len(final_results_df))
final_results_df = final_results_df.sort_values(by=["adj.P.Val"])

res = [] 
final_results_list2 = []
for index, row in final_results_df.iterrows():
  expressed_gene = row['Gene.symbol']
  if expressed_gene not in res: 
        final_results_list2.append(row) 
        res.append(expressed_gene)

comparative_analysis_results = pd.DataFrame(final_results_list2)
comparative_analysis_results = comparative_analysis_results[["ID","adj.P.Val","P.Value","t","B","logFC", "Gene.symbol"]]
print(len(final_results_list2))
print(res)
comparative_analysis_results.to_csv("Comparative_Analysis_Results.csv")




18
{'BACH2', 'HIPK2', 'STAT1', 'KDM4B', 'KLHL9', 'RNASEH2C', 'ETS1', 'UBE2L3', 'SMARCD1', 'IKZF1', 'CD40', 'IRF7', 'ABCA1', 'LYST', 'JAK2', 'LGALS9', 'RNF114', 'USP18'}
91
18
['JAK2', 'IRF7', 'RNF114', 'IKZF1', 'STAT1', 'HIPK2', 'KDM4B', 'ETS1', 'USP18', 'RNASEH2C', 'LGALS9', 'LYST', 'BACH2', 'UBE2L3', 'CD40', 'ABCA1', 'SMARCD1', 'KLHL9']


Here we will find the list of genes that overlap from KEGG pathways, Case vs Control analysis, and the eQTL Analysis. These will be studied further in our Literature analysis. 

In [ ]:
import pandas as pd
import os
Kegg_Genes = pd.read_csv("KEGG_interesting_mapped.csv")

Kegg_genes_list = Kegg_Genes["geneSymbol"]
res_set = set(res)
kegg_set = set(Kegg_genes_list)
final_list_overlap = res_set.intersection(kegg_set)
print(final_list_overlap)




{'BACH2', 'KDM4B', 'ETS1', 'UBE2L3', 'CD40', 'IKZF1', 'LYST', 'JAK2', 'HIPK2'}


Not using this code or below. 

In [ ]:
import pandas as pd
import os
df = pd.read_csv('original_data.csv') 
df.head()
df = df[['SNPS','UPSTREAM_GENE_ID','DOWNSTREAM_GENE_ID','SNP_GENE_IDS','CONTEXT','MAPPED_TRAIT']]
df.head()


upStreamList = df.UPSTREAM_GENE_ID.tolist()
geneNames = []

f = open("upStreamGeneNames.txt", "a")
for item in upStreamList:
  if str(item) == 'nan':
    pass
  else: 
    if (len(str(item)) > 16):
      gene_IDS = str(item)
      gene_IDS = gene_IDS.split(", ")
      for thing in gene_IDS:

        f.write(str(thing) + "\n")
    else: 
      f.write(str(item) + "\n")

f.close()






In [ ]:
import pandas as pd
import os
df = pd.read_csv('original_data.csv') 
df.head()
df = df[['SNPS','UPSTREAM_GENE_ID','DOWNSTREAM_GENE_ID','SNP_GENE_IDS','CONTEXT','MAPPED_TRAIT']]
df.head()


downStreamList = df.DOWNSTREAM_GENE_ID.tolist()
geneNames = []

f = open("downStreamGeneNames.txt", "a")
for item in downStreamList:
  if str(item) == 'nan':
    pass
  else: 
    if (len(str(item)) > 16):
      gene_IDS = str(item)
      gene_IDS = gene_IDS.split(", ")
      for thing in gene_IDS:

        f.write(str(thing) + "\n")
    else: 
      f.write(str(item) + "\n")

f.close()




